# Mathematik für Biologiestudierende II

Sommersemester 2024

30.04.2024

&copy; 2024 Prof. Dr. Rüdiger W. Braun 

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
sns.set_theme()

# Mutiples Testen

### Beispiel Gummibärchen

![Cartoon aus xkcd](https://imgs.xkcd.com/comics/significant.png)

Quelle: https://xkcd.com/882

* Ein Fall von Data Snooping
* Bei einem Signifikanztest zum Nivea $\alpha=0.05$ wird in 5% der Fälle die Nullhypothese fälschlich abgelehnt
* In dem Beispiel des Cartoons gibt es 20 Experimente; es ist zu erwarten, dass in einem Fall die Nullhypothese zu unrecht abgelehnt wird

## Multiple Vergleiche

Möglichkeiten für korrektes Vorgehen
 

Man testet die Nullhypothese 

> $H_0$ = "Grüne Gummibärchen verursachen keine Akne"

in einem neuen Experiment zum Signifikanzniveau~5\%

oder man rechnet des multiple Experiment mit einer Korrektur des Signifikanzniveaus

* Bonferroni-Korrektur
* Bonferroni-Holm-Korrektur
* False Discovery Rate

## Bonferroni-Korrektur

* Wenn man simultan $n$ Vergleiche durchführt, dann schreibt die Bonferroni-Korrektur vor, dass man jeden einzelnen Vergleich zum Signifikanzniveau $\frac\alpha n$ durchführt
* Im Beispiel der Gummibärchen hätten die Einzelversuche zum Signifikanzniveau $\frac\alpha{20} = 0.0025$ durchgeführt werden müssen

* Im Beispiel der Schwarzstörche hätte für jeden Einzeltest das Signifikanzniveau $\alpha = \frac{0.05}{100} =  0.0005$ gewählt werden müssen

## Bonferroni-Holm-Korrektur 

erkläre ich, nachdem ich die Bonferroni-Korrektur für die ANOVA vorgemacht habe

## False Discovery Rate

* Beispiel Bilddaten: 20 MNR-Aufnahmen von gesunden Gehirnen und 20 MNR-Aufnahmen von erkrankten Gehirnen, wobei die Gehirne auf einen Standardatlas normalisiert werden
* In einem Bild werden alle Voxel (3D-Pixel) markiert, bei denen der Eisengehalt der Gruppe der Erkrankten signifikant über dem der Gruppe der Gesunden liegt

*  Millionen von parallelen Experimenten:  Bonferroni-Korrektur praktisch unmöglich
* Alternative: False Discovery Rate
* FDR von 1% sagt: im Schnitt sind nur 1% aller markierten Voxel falsch
* Zum Vergleich: Ein multipler Test zum Signifikanzniveau 5% sagt: Die Wahrscheinlichkeit, dass auch nur ein einziges Voxel zu Unrecht markiert ist, beträgt höchstens 5%

# ANOVA

Zurück zu den Pinguinen

In [2]:
df = sns.load_dataset("penguins") 
gA = df[df.species=='Adelie'].bill_length_mm
gG = df[df.species=='Gentoo'].bill_length_mm
gC = df[df.species=='Chinstrap'].bill_length_mm

In [3]:
res = stats.f_oneway(gA.dropna(), gG.dropna(), gC.dropna())
res

F_onewayResult(statistic=410.6002550405077, pvalue=2.6946137388895484e-91)

Die Schnabellängen unterscheiden sich also

Wir hätten auch drei t-Tests rechnen können

In [4]:
r1 = stats.ttest_ind(gA.dropna(), gG.dropna())
r1

TtestResult(statistic=-25.09530115900974, pvalue=9.324042980315958e-73, df=272.0)

In [11]:
r2 = stats.ttest_ind(gA.dropna(), gC.dropna())
r2

TtestResult(statistic=-23.801939237440887, pvalue=2.011759018655462e-62, df=217.0)

In [12]:
r3 = stats.ttest_ind(gG.dropna(), gC.dropna())
r3

TtestResult(statistic=-2.7694045269151144, pvalue=0.006175813141889592, df=189.0)

* Das ist multiples Testen, muss also korrigiert werden

* Drei Tests gerechnet
* Gewünscht: $\alpha=0.01$
* Bonferroni-Korrektur:  Jeden einzelnen Test zu $\frac\alpha3 = 0.003333$ auswerten

* Zu $\alpha=0.01$ werden Unterschiede in den Schnabellängen zwischen Adelie- und Eselspinguinen und zwischen Adelie- und Zügelpinguinen gefunden
* Der Unterschied zwischen Esels- und Zügelpinguinen ist nicht signifikant

# Post-hoc Analyse

* Wenn die ANOVA einen signifikanten Unterschied zwischen den Gruppen gezeigt hat, dann versucht man mit der post-hoc Analyise herauszubekommen, zwischen welchen beiden Gruppen signifikante Unterschiede bestehen
* Die post-hoc Analyse muss für multiple Vergleiche korrigiert werden

In [31]:
import statsmodels.sandbox.stats.multicomp as mc

Achtung:  Hier wird irgendwann der Bestandteil `sandbox` überflüssig

In [32]:
mc.multipletests([r1.pvalue, r2.pvalue, r3.pvalue], alpha=0.01, method='bonferroni')

(array([ True,  True, False]),
 array([2.79721289e-72, 6.03527706e-62, 1.85274394e-02]),
 0.003344506587403595,
 0.0033333333333333335)

### Dasselbe mit Bonferroni-Holm

In [33]:
mc.multipletests([r1.pvalue, r2.pvalue, r3.pvalue], alpha=0.01, method='holm')

(array([ True,  True,  True]),
 array([2.79721289e-72, 4.02351804e-62, 6.17581314e-03]),
 0.003344506587403595,
 0.0033333333333333335)

## Bonferroni-Holm

* n multiple Vergleiche werden durchgeführt
* Die p-Werte werden der Größe nach geordnet
* der kleinste p-Wert muss signifikant zu $\frac\alpha n$ sein
* der zweitkleinste zu $\frac\alpha{n-1}$
* drittkleinste zu $\frac\alpha{n-2}$
* usw.
* der größte zum Niveau $\alpha$

Bei den Pinguinen

* der kleinste p-Wert ist 2.79721289e-72, er ist kleiner als $\frac{0.01}3 = 0.003333$
* der zweitkleinste p-Wert ist 4.02351804e-62, er ist kleiner als $\frac{0.01}2 = 0.005$
* der drittkleinste p-Wert ist 0.006175813142, er ist kleiner als $\alpha = 0.01$


* Wendet man Bonferroni-Holm an, so unterscheiden sich die auch Schnabellängen zwischen Esels- und Zügelpinguinen signifikant

* Genau wie die Wahl des Signifikanzniveaus muss die Frage nach der Korrektur des p-Werts innerhalb der Fachwissenschaft beantwortet werden